In [35]:
#Standard Lib imports
import sys, os
from pathlib import Path
#from functools import partial
import getpass

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))


# Third party imports

# Local application imports
from sei import sei

#USR = getpass.getuser()
USR = 'rsilva'
print(f'Olá {USR}')
PWD = getpass.getpass("Digite sua Senha: ")

%load_ext autoreload
%autoreload 2

Olá rsilva
Digite sua Senha: ········
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
s = sei.login_sei(USR, PWD, browser='Firefox', timeout=5, teste=False)

In [37]:
s.mudar_lotação("GR01")

In [38]:
s.mudar_lotação("GR01FI3")

In [39]:
s.mudar_lotação("GR01AF")

Não existe a opção GR01AF no Menu mencionado


In [ ]:
s.mudar_lotação("GR01OR")

In [8]:
s.ver_detalhado()

In [9]:
s.is_init_page()

True

In [10]:
s.go_to_init_page()

In [12]:
s.show_lat_menu()

In [13]:
s.page.timeout = 2

In [14]:
s.ver_detalhado()

In [15]:
s.ver_todos()

In [17]:
s.itera_processos()

In [18]:
s.get_processos()

OrderedDict([('53500.201054/2014-20',
              {'checkbox': <input class="infraCheckbox" id="chkDetalhadoItem0" name="chkDetalhadoItem0" onclick="infraSelecionarItens(this,'Detalhado');" title="53500.201054/2014-20" type="checkbox" value="6830"/>,
               'aviso': '',
               'link': 'https://sei.anatel.gov.br/sei/controlador.php?acao=procedimento_trabalhar&acao_origem=procedimento_controlar&acao_retorno=procedimento_controlar&id_procedimento=6830&infra_sistema=100000100&infra_unidade_atual=110001002&infra_hash=c351be0bee702bfc8887fc8fa2143a77c7854e466af760edf1f8c83a30293134',
               'numero': '53500.201054/2014-20',
               'visualizado': False,
               'atribuicao': 'amorim',
               'tipo': 'Fiscalização: Certificação de Produtos (desativado)',
               'interessado': 'Ministério Público Estadual de Pernambuco'}),
             ('53504.000031/2019-74',
              {'checkbox': <input class="infraCheckbox" id="chkDetalhadoItem1" 

In [19]:
s.go_to_blocos()

In [64]:
num = "53504.005855/2019-31"
p = s.go_to_processo(num)

In [61]:
p.is_open("GR01FI3")

True

In [65]:
p.is_open("GR01OR")

True

In [73]:
p.concluir_processo()

In [75]:
p.abrir_processo()

In [66]:
p._click_na_arvore('5023038')

In [ ]:
p.incluir_documento('Externo')

In [ ]:
p.page.timeout = 2

In [70]:
p.send_doc_por_email('5023038', ('', "Teste", "Medição RFeye"))

TimeoutException: Message: 


In [ ]:
p.abrir_pastas()

In [ ]:
p._get_acoes('0177509')

In [ ]:
p.is_open()